In [1]:
import pandas as pd

soil = pd.read_csv(r"D:\S3\data_mining\projet\soil_data.csv")
soil.head()

,HWSD2_SMU_ID,longitude,latitude,COARSE,SAND,SILT,CLAY,TEXTURE_USDA,TEXTURE_SOTER,BULK,...,CEC_SOIL,CEC_CLAY,CEC_EFF,TEB,BSAT,ALUM_SAT,ESP,TCARBON_EQ,GYPSUM,ELEC_COND
0,31802,8.937500,37.537500,11,40,41,19,9.0,M,1.47,...,11,26,6.0,5.0,47,21,1,0.0,0.1,0
1,31802,8.937500,37.537500,9,25,53,22,7.0,M,1.46,...,15,51,14.0,12.0,65,26,3,0.0,0.1,0
2,31805,8.945833,37.537500,9,87,9,4,12.0,C,1.44,...,4,33,3.0,3.0,76,6,2,0.0,0.1,0
3,31805,8.945833,37.537500,5,33,31,36,5.0,F,1.35,...,20,34,15.0,17.0,82,0,1,0.0,1.6,1
4,31802,8.912500,37.529167,11,40,41,19,9.0,M,1.47,...,11,26,6.0,5.0,47,21,1,0.0,0.1,0


In [3]:
import rasterio
import pandas as pd
from rasterio.transform import rowcol
import numpy as np
import glob

# --- 1. DEFINE FILE PATHS (CRITICAL: UPDATE THESE) ---
# Assuming you saved your final climate/fire data here:
climate_fire_file = r"D:\S3\rapport data_mining\newdata\fire_climate_7km_final.csv"
# **UPDATE THIS PATH** to the location of your soil data CSV
soil_file = r"D:\S3\data_mining\projet\soil_data.csv" 
# This path is needed to get the grid transform
tif_folder = r"D:\S3\data_mining\projet\clipped_imputed_data2\\" 
tif_files = sorted(glob.glob(tif_folder + "*_imputed_local.tif"))


# --- 2. LOAD DATA ---
climate_fire_df = pd.read_csv(climate_fire_file)
#soil_df = pd.read_csv(soil_file)

# Get the Transformation matrix from the first climate TIF file
with rasterio.open(tif_files[0]) as src:
    transform = src.transform

# --- 3. MAP SOIL COORDINATES TO GRID INDICES ---
# Calculate 'row' and 'col' for the soil data using the TIF transform
rows, cols = rowcol(transform, soil['longitude'], soil['latitude'])
soil['row'] = rows
soil['col'] = cols

# --- 4. PREPARE SOIL DATA FOR MERGE ---
# Keep all soil property columns and drop the redundant float coordinates
soil_cols_to_keep = [
    col for col in soil.columns 
    if col not in ['longitude', 'latitude']
]
# Ensure we only have unique grid cells for merging
soil_df_final = soil[soil_cols_to_keep].drop_duplicates(subset=['row', 'col'])


# --- 5. PERFORM THE FINAL MERGE ---
# Use a LEFT JOIN to keep all rows in the climate_fire_df (all your land pixels)
# and add the soil properties.
final_combined_df = climate_fire_df.merge(
    soil_df_final,
    on=['row', 'col'],
    how='left'
)

# --- 6. CLEANUP AND SAVE ---
# Check for missing soil data (where a land pixel didn't match a soil polygon)
print(f"Final Combined DataFrame size: {len(final_combined_df)}")
print(f"Number of land pixels with missing soil data: {final_combined_df['HWSD2_SMU_ID'].isna().sum()}")

# Fill remaining NaN values in soil columns (optional, based on model needs)
# You may want to fill soil properties with 0 or the mean/median, depending on the variable.
# For now, we'll keep the NaNs or use a simple fill for demonstration:
# soil_property_cols = [col for col in final_combined_df.columns if col.upper() in ['COARSESAND', 'SILT', 'CLAY', 'BULK']]
# final_combined_df[soil_property_cols] = final_combined_df[soil_property_cols].fillna(0)


# Save the final, complete dataset
final_combined_df.to_csv("final_climate_fire_soil_data.csv", index=False)

final_combined_df.head()

KeyError: 'row'

In [4]:
"""
Complete Climate + Fire + Soil Data Integration
================================================
This script merges climate data (7km grid), fire occurrences, and soil properties
into a single ML-ready dataset.

Author: Climate Analysis Pipeline
Date: 2025

deleate itttttttttttttttt
"""

import rasterio
import pandas as pd
from rasterio.transform import rowcol
import numpy as np
import glob
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# CONFIGURATION - UPDATE THESE PATHS
# ============================================================================

# Input files
CLIMATE_FIRE_FILE = r"D:\S3\rapport data_mining\newdata\fire_climate_7km_final.csv"
SOIL_FILE = r"D:\S3\data_mining\projet\soil_data.csv"
TIF_FOLDER = r"D:\S3\data_mining\projet\clipped_imputed_data2\\"

# Output file
OUTPUT_FILE = "final_climate_fire_soil_7km.csv"

# Options
FILL_MISSING_SOIL = True  # Fill missing soil values with median?
DROP_NO_SOIL = False      # Drop rows without soil data? (Not recommended)


# ============================================================================
# MAIN PROCESSING
# ============================================================================

def main():
    """
    Main function to merge climate, fire, and soil data.
    """
    
    print("="*70)
    print("  CLIMATE + FIRE + SOIL DATA INTEGRATION")
    print("="*70)
    
    # ------------------------------------------------------------------------
    # STEP 1: LOAD DATA
    # ------------------------------------------------------------------------
    print("\n📂 STEP 1: LOADING DATA")
    print("-" * 70)
    
    # Load climate + fire data
    print(f"Loading climate/fire data from: {CLIMATE_FIRE_FILE}")
    climate_fire_df = pd.read_csv(CLIMATE_FIRE_FILE)
    print(f"  ✅ Loaded {len(climate_fire_df):,} rows × {len(climate_fire_df.columns)} columns")
    
    # Load soil data
    print(f"\nLoading soil data from: {SOIL_FILE}")
    soil_df = pd.read_csv(SOIL_FILE)
    print(f"  ✅ Loaded {len(soil_df):,} rows × {len(soil_df.columns)} columns")
    
    # Get TIF files for transform matrix
    tif_files = sorted(glob.glob(TIF_FOLDER + "*.tif"))
    if len(tif_files) == 0:
        raise ValueError(f"❌ No TIF files found in {TIF_FOLDER}")
    
    print(f"\nLoading transform matrix from: {tif_files[0]}")
    with rasterio.open(tif_files[0]) as src:
        transform = src.transform
    print(f"  ✅ Transform matrix loaded")
    
    
    # ------------------------------------------------------------------------
    # STEP 2: CALCULATE GRID INDICES FOR CLIMATE DATA
    # ------------------------------------------------------------------------
    print("\n🗺️  STEP 2: CALCULATING GRID INDICES FOR CLIMATE DATA")
    print("-" * 70)
    
    print("Converting (longitude, latitude) to (row, col) for climate data...")
    climate_rows, climate_cols = rowcol(
        transform, 
        climate_fire_df['longitude'].values, 
        climate_fire_df['latitude'].values
    )
    
    climate_fire_df['row'] = climate_rows
    climate_fire_df['col'] = climate_cols
    
    print(f"  ✅ Added 'row' and 'col' columns to climate data")
    print(f"     Row range: {climate_fire_df['row'].min()} to {climate_fire_df['row'].max()}")
    print(f"     Col range: {climate_fire_df['col'].min()} to {climate_fire_df['col'].max()}")
    
    
    # ------------------------------------------------------------------------
    # STEP 3: CALCULATE GRID INDICES FOR SOIL DATA
    # ------------------------------------------------------------------------
    print("\n🌱 STEP 3: CALCULATING GRID INDICES FOR SOIL DATA")
    print("-" * 70)
    
    print("Converting (longitude, latitude) to (row, col) for soil data...")
    soil_rows, soil_cols = rowcol(
        transform, 
        soil_df['longitude'].values, 
        soil_df['latitude'].values
    )
    
    soil_df['row'] = soil_rows
    soil_df['col'] = soil_cols
    
    print(f"  ✅ Added 'row' and 'col' columns to soil data")
    print(f"     Row range: {soil_df['row'].min()} to {soil_df['row'].max()}")
    print(f"     Col range: {soil_df['col'].min()} to {soil_df['col'].max()}")
    
    
    # ------------------------------------------------------------------------
    # STEP 4: PREPARE SOIL DATA FOR MERGE
    # ------------------------------------------------------------------------
    print("\n🔧 STEP 4: PREPARING SOIL DATA FOR MERGE")
    print("-" * 70)
    
    # Get all soil columns except the float coordinates
    soil_cols_to_keep = [
        col for col in soil_df.columns 
        if col not in ['longitude', 'latitude']
    ]
    
    print(f"Keeping {len(soil_cols_to_keep)} soil columns")
    
    # Remove duplicate grid cells (keep first occurrence)
    soil_df_unique = soil_df[soil_cols_to_keep].drop_duplicates(subset=['row', 'col'])
    
    duplicates_removed = len(soil_df) - len(soil_df_unique)
    print(f"  ✅ Removed {duplicates_removed:,} duplicate grid cells")
    print(f"  ✅ Unique soil grid cells: {len(soil_df_unique):,}")
    
    # Identify soil property columns
    soil_property_cols = [
        col for col in soil_df_unique.columns 
        if col not in ['row', 'col'] and not col.startswith('Unnamed')
    ]
    print(f"  ✅ Soil properties to merge: {len(soil_property_cols)}")
    if len(soil_property_cols) <= 10:
        print(f"     {', '.join(soil_property_cols)}")
    else:
        print(f"     {', '.join(soil_property_cols[:10])}...")
    
    
    # ------------------------------------------------------------------------
    # STEP 5: MERGE CLIMATE + FIRE + SOIL
    # ------------------------------------------------------------------------
    print("\n🔗 STEP 5: MERGING DATASETS")
    print("-" * 70)
    
    print("Performing LEFT JOIN on (row, col)...")
    print("  Strategy: Keep all climate points, add soil where available")
    
    final_df = climate_fire_df.merge(
        soil_df_unique,
        on=['row', 'col'],
        how='left'
    )
    
    print(f"  ✅ Merge complete!")
    print(f"     Input rows: {len(climate_fire_df):,}")
    print(f"     Output rows: {len(final_df):,}")
    print(f"     Columns: {len(climate_fire_df.columns)} → {len(final_df.columns)}")
    
    
    # ------------------------------------------------------------------------
    # STEP 6: ANALYZE DATA COVERAGE
    # ------------------------------------------------------------------------
    print("\n📊 STEP 6: ANALYZING DATA COVERAGE")
    print("-" * 70)
    
    # Check soil data coverage
    if soil_property_cols:
        first_soil_col = soil_property_cols[0]
        missing_soil = final_df[first_soil_col].isna().sum()
        has_soil = len(final_df) - missing_soil
        
        print(f"Soil Data Coverage:")
        print(f"  ✅ Points WITH soil data:    {has_soil:,} ({100*has_soil/len(final_df):.1f}%)")
        print(f"  ❌ Points WITHOUT soil data: {missing_soil:,} ({100*missing_soil/len(final_df):.1f}%)")
        
        # Check missing values per soil column
        print(f"\nMissing values by soil property:")
        for col in soil_property_cols[:10]:  # Show first 10
            missing = final_df[col].isna().sum()
            print(f"  {col:30s}: {missing:,} missing ({100*missing/len(final_df):.1f}%)")
        
        if len(soil_property_cols) > 10:
            print(f"  ... and {len(soil_property_cols) - 10} more soil properties")
    
    # Column breakdown
    climate_cols = [c for c in final_df.columns if any(x in c for x in ['tmin', 'tmax', 'prec', 'tavg'])]
    
    print(f"\nColumn Breakdown:")
    print(f"  Climate variables: {len(climate_cols)}")
    print(f"  Soil properties: {len(soil_property_cols)}")
    print(f"  Other (coords, indices, fire): {len(final_df.columns) - len(climate_cols) - len(soil_property_cols)}")
    
    # Fire distribution
    if 'is_fire' in final_df.columns:
        fire_count = (final_df['is_fire'] == 1).sum()
        print(f"\nFire Distribution:")
        print(f"  🔥 Fire points: {fire_count:,} ({100*fire_count/len(final_df):.2f}%)")
        print(f"  ⭕ Non-fire points: {len(final_df) - fire_count:,} ({100*(len(final_df)-fire_count)/len(final_df):.2f}%)")
    
    
    # ------------------------------------------------------------------------
    # STEP 7: HANDLE MISSING VALUES
    # ------------------------------------------------------------------------
    print("\n🔧 STEP 7: HANDLING MISSING VALUES")
    print("-" * 70)
    
    if DROP_NO_SOIL:
        print("Option: DROP rows without soil data")
        before = len(final_df)
        final_df = final_df.dropna(subset=soil_property_cols)
        after = len(final_df)
        print(f"  ✅ Dropped {before - after:,} rows")
        print(f"  ✅ Remaining: {after:,} rows")
        
    elif FILL_MISSING_SOIL:
        print("Option: FILL missing soil values with median")
        filled_count = 0
        
        for col in soil_property_cols:
            if final_df[col].isna().any():
                # Check if column is numeric
                if pd.api.types.is_numeric_dtype(final_df[col]):
                    median_val = final_df[col].median()
                    final_df[col].fillna(median_val, inplace=True)
                    filled_count += 1
                    print(f"  Filled '{col}' with median: {median_val:.2f}")
                else:
                    # For categorical, fill with mode
                    mode_val = final_df[col].mode()[0] if not final_df[col].mode().empty else 'Unknown'
                    final_df[col].fillna(mode_val, inplace=True)
                    filled_count += 1
                    print(f"  Filled '{col}' with mode: {mode_val}")
        
        print(f"  ✅ Filled {filled_count} soil columns")
    else:
        print("Option: KEEP missing values as NaN")
        print("  ⚠️  Warning: Some ML algorithms may not handle NaN values")
    
    
    # ------------------------------------------------------------------------
    # STEP 8: FINAL VALIDATION
    # ------------------------------------------------------------------------
    print("\n✅ STEP 8: FINAL VALIDATION")
    print("-" * 70)
    
    # Check for any remaining NaN values
    total_nan = final_df.isna().sum().sum()
    if total_nan > 0:
        print(f"⚠️  Warning: {total_nan:,} NaN values remain in dataset")
        cols_with_nan = final_df.columns[final_df.isna().any()].tolist()
        print(f"   Columns with NaN: {', '.join(cols_with_nan[:5])}...")
    else:
        print(f"✅ No missing values - dataset is complete!")
    
    # Check data types
    print(f"\nData Types:")
    print(f"  Numeric columns: {len(final_df.select_dtypes(include=[np.number]).columns)}")
    print(f"  Object columns: {len(final_df.select_dtypes(include=['object']).columns)}")
    
    # Memory usage
    memory_mb = final_df.memory_usage(deep=True).sum() / 1024**2
    print(f"\nMemory Usage: {memory_mb:.1f} MB")
    
    
    # ------------------------------------------------------------------------
    # STEP 9: SAVE FINAL DATASET
    # ------------------------------------------------------------------------
    print("\n💾 STEP 9: SAVING FINAL DATASET")
    print("-" * 70)
    
    print(f"Saving to: {OUTPUT_FILE}")
    final_df.to_csv(OUTPUT_FILE, index=False)
    
    file_size_mb = pd.read_csv(OUTPUT_FILE).memory_usage(deep=True).sum() / 1024**2
    print(f"  ✅ File saved successfully!")
    print(f"     Rows: {len(final_df):,}")
    print(f"     Columns: {len(final_df.columns)}")
    print(f"     File size: ~{file_size_mb:.1f} MB")
    
    
    # ------------------------------------------------------------------------
    # FINAL SUMMARY
    # ------------------------------------------------------------------------
    print("\n" + "="*70)
    print("🎉 PROCESSING COMPLETE!")
    print("="*70)
    
    print(f"\n📋 FINAL DATASET SUMMARY:")
    print(f"   Total grid points: {len(final_df):,}")
    print(f"   Total features: {len(final_df.columns)}")
    
    if 'is_fire' in final_df.columns:
        fire_count = (final_df['is_fire'] == 1).sum()
        print(f"   Fire points: {fire_count:,} ({100*fire_count/len(final_df):.2f}%)")
    
    print(f"\n📊 FEATURE BREAKDOWN:")
    print(f"   Climate variables: {len(climate_cols)}")
    print(f"   Soil properties: {len(soil_property_cols)}")
    print(f"   Geographic: 4 (lon, lat, row, col)")
    print(f"   Target: 1 (is_fire)")
    
    print(f"\n💾 OUTPUT FILE: {OUTPUT_FILE}")
    print(f"\n✨ Your dataset is ready for machine learning!")
    
    return final_df


# ============================================================================
# PREVIEW FUNCTION
# ============================================================================

def preview_dataset(df, n_rows=5):
    """
    Display a preview of the dataset.
    """
    print("\n" + "="*70)
    print("📋 DATASET PREVIEW")
    print("="*70)
    
    print(f"\nFirst {n_rows} rows:")
    print(df.head(n_rows).to_string())
    
    print(f"\n\nAll columns ({len(df.columns)}):")
    for i, col in enumerate(df.columns, 1):
        print(f"  {i:2d}. {col}")
    
    print(f"\n\nBasic Statistics:")
    print(df.describe().to_string())


# ============================================================================
# ENTRY POINT
# ============================================================================

if __name__ == "__main__":
    print("""
    ╔══════════════════════════════════════════════════════════════════╗
    ║                                                                  ║
    ║   CLIMATE + FIRE + SOIL DATA INTEGRATION PIPELINE               ║
    ║                                                                  ║
    ║   Combines 7km climate grid, fire occurrences, and soil data   ║
    ║   into a single ML-ready dataset                                ║
    ║                                                                  ║
    ╚══════════════════════════════════════════════════════════════════╝
    """)
    
    print(f"\n📋 CONFIGURATION:")
    print(f"   Input files:")
    print(f"     - Climate/Fire: {CLIMATE_FIRE_FILE}")
    print(f"     - Soil: {SOIL_FILE}")
    print(f"     - TIF folder: {TIF_FOLDER}")
    print(f"\n   Output file: {OUTPUT_FILE}")
    print(f"\n   Options:")
    print(f"     - Fill missing soil: {FILL_MISSING_SOIL}")
    print(f"     - Drop rows without soil: {DROP_NO_SOIL}")
    print(f"\n{'='*70}\n")
    
    try:
        # Run main processing
        final_df = main()
        
        # Show preview
        preview_dataset(final_df, n_rows=5)
        
        print("\n" + "="*70)
        print("SUCCESS! 🎉")
        print("="*70)
        
    except Exception as e:
        print(f"\n❌ ERROR: {str(e)}")
        import traceback
        traceback.print_exc()
        print("\n💡 TIP: Check that all file paths are correct and files exist!")


    ╔══════════════════════════════════════════════════════════════════╗
    ║                                                                  ║
    ║   CLIMATE + FIRE + SOIL DATA INTEGRATION PIPELINE               ║
    ║                                                                  ║
    ║   Combines 7km climate grid, fire occurrences, and soil data   ║
    ║   into a single ML-ready dataset                                ║
    ║                                                                  ║
    ╚══════════════════════════════════════════════════════════════════╝
    

📋 CONFIGURATION:
   Input files:
     - Climate/Fire: D:\S3\rapport data_mining\newdata\fire_climate_7km_final.csv
     - Soil: D:\S3\data_mining\projet\soil_data.csv
     - TIF folder: D:\S3\data_mining\projet\clipped_imputed_data2\\

   Output file: final_climate_fire_soil_7km.csv

   Options:
     - Fill missing soil: True
     - Drop rows without soil: False


  CLIMATE + FIRE + SOIL DATA INTEGRATION

📂 ST

,row,col,longitude,latitude,tmin_2024_1,tmin_2024_2,tmin_2024_3,tmin_2024_4,tmin_2024_5,tmin_2024_6,...,CEC_SOIL,CEC_CLAY,CEC_EFF,TEB,BSAT,ALUM_SAT,ESP,TCARBON_EQ,GYPSUM,ELEC_COND
0,0,144,3.375000,37.291667,8.250000,8.250000,10.0000,11.500000,15.000,18.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,145,3.458333,37.291667,8.250000,8.250000,10.1250,11.750000,15.000,18.125000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,146,3.541667,37.291667,8.333333,8.333333,10.2500,11.666667,15.000,18.166666,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,147,3.625000,37.291667,8.250000,8.250000,10.0625,11.562500,14.875,18.062500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,148,3.708333,37.291667,8.250000,8.250000,10.1000,11.550000,14.800,18.150000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,6,195,7.625000,36.791667,9.000000,9.000000,10.0000,11.750000,15.000,18.500000,...,18.0,41.0,18.0,17.0,83.0,0.0,4.0,0.0,0.0,1.0
996,6,195,7.625000,36.791667,9.000000,9.000000,10.0000,11.750000,15.000,18.500000,...,18.0,41.0,18.0,17.0,83.0,0.0,4.0,0.0,0.0,1.0
997,6,195,7.625000,36.791667,9.000000,9.000000,10.0000,11.750000,15.000,18.500000,...,18.0,41.0,18.0,17.0,83.0,0.0,4.0,0.0,0.0,1.0
998,6,195,7.625000,36.791667,9.000000,9.000000,10.0000,11.750000,15.000,18.500000,...,18.0,41.0,18.0,17.0,83.0,0.0,4.0,0.0,0.0,1.0


hereeeeeeeeeeeeeeeeeeeeeeeeeeeeee

In [3]:
import pandas as pd
import glob

# --- SIMPLER VERSION: MERGE DIRECTLY ON COORDINATES ---
climate_fire_file = r"D:\S3\data_mining\projet2\final_dataset.csv"
soil_file = r"D:\S3\data_mining\projet\soil_data.csv" 

# Load data
climate_fire_df = pd.read_csv(climate_fire_file)
soil_df = pd.read_csv(soil_file)

# Round coordinates to avoid floating point precision issues
climate_fire_df['longitude'] = climate_fire_df['longitude'].round(6)
climate_fire_df['latitude'] = climate_fire_df['latitude'].round(6)
soil_df['longitude'] = soil_df['longitude'].round(6)
soil_df['latitude'] = soil_df['latitude'].round(6)

print(f"Climate fire data points: {len(climate_fire_df)}")
print(f"Soil data points: {len(soil_df)}")

# Merge directly on coordinates
final_combined_df = climate_fire_df.merge(
    soil_df,
    on=['longitude', 'latitude'],
    how='left'
)

print(f"Final combined data points: {len(final_combined_df)}")
print(f"Missing soil data: {final_combined_df['HWSD2_SMU_ID'].isna().sum()}")

# Save result
final_combined_df.to_csv(r"D:\S3\data_mining\projet\final_climate_fire_soil_data.csv", index=False)
print("Dataset saved successfully!")

Climate fire data points: 807777
Soil data points: 7113536
Final combined data points: 807777
Missing soil data: 807777
Dataset saved successfully!


In [5]:
import rasterio
import pandas as pd
from rasterio.transform import rowcol
import numpy as np
import glob

# --- 1. DEFINE FILE PATHS ---
climate_fire_file = r"D:\S3\data_mining\projet2\final_dataset.csv"
soil_file = r"D:\S3\data_mining\projet\soil_data.csv" 
tif_folder = r"D:\S3\data_mining\projet\clipped_imputed_data2\\" 
tif_files = sorted(glob.glob(tif_folder + "*_imputed_local.tif"))

# --- 2. LOAD DATA ---
climate_fire_df = pd.read_csv(climate_fire_file)
soil_df = pd.read_csv(soil_file)

# Get the Transformation matrix from the first climate TIF file
with rasterio.open(tif_files[0]) as src:
    transform = src.transform

# --- 3. ADD ROW AND COL TO CLIMATE_FIRE_DF ---
# Calculate 'row' and 'col' for the climate_fire data using the TIF transform
print("Adding row and col to climate_fire data...")
rows, cols = rowcol(transform, climate_fire_df['longitude'], climate_fire_df['latitude'])
climate_fire_df['row'] = rows
climate_fire_df['col'] = cols

# --- 4. MAP SOIL COORDINATES TO GRID INDICES ---
# Calculate 'row' and 'col' for the soil data using the TIF transform
print("Adding row and col to soil data...")
rows, cols = rowcol(transform, soil_df['longitude'], soil_df['latitude'])
soil_df['row'] = rows
soil_df['col'] = cols

# --- 5. PREPARE SOIL DATA FOR MERGE ---
# Keep all soil property columns and drop the redundant float coordinates
soil_cols_to_keep = [
    col for col in soil_df.columns 
    if col not in ['longitude', 'latitude']
]
# Ensure we only have unique grid cells for merging
soil_df_final = soil_df[soil_cols_to_keep].drop_duplicates(subset=['row', 'col'])

print(f"Climate fire data shape: {climate_fire_df.shape}")
print(f"Soil data shape: {soil_df_final.shape}")
print(f"Climate fire unique grid cells: {climate_fire_df[['row', 'col']].drop_duplicates().shape[0]}")
print(f"Soil unique grid cells: {soil_df_final[['row', 'col']].drop_duplicates().shape[0]}")

# --- 6. PERFORM THE FINAL MERGE ---
# Use a LEFT JOIN to keep all rows in the climate_fire_df
final_combined_df = climate_fire_df.merge(
    soil_df_final,
    on=['row', 'col'],
    how='left'
)

# --- 7. CLEANUP AND SAVE ---
print(f"Final Combined DataFrame size: {len(final_combined_df)}")
print(f"Number of pixels with missing soil data: {final_combined_df['HWSD2_SMU_ID'].isna().sum()}")

# Check which columns have missing data
missing_data = final_combined_df.isnull().sum()
print("\nMissing data per column:")
print(missing_data[missing_data > 0])

# Save the final, complete dataset
final_combined_df.to_csv(r"D:\S3\data_mining\projet\final_climate_fire_soil_data.csv", index=False)
print("Final dataset saved successfully!")

Adding row and col to climate_fire data...
Adding row and col to soil data...
Climate fire data shape: (698106, 41)
Soil data shape: (33476, 25)
Climate fire unique grid cells: 33396
Soil unique grid cells: 33476
Final Combined DataFrame size: 698106
Number of pixels with missing soil data: 1532

Missing data per column:
HWSD2_SMU_ID       1532
COARSE             1532
SAND               1532
SILT               1532
CLAY               1532
TEXTURE_USDA     168565
TEXTURE_SOTER      1532
BULK               1532
REF_BULK           1532
ORG_CARBON         1532
PH_WATER           1532
TOTAL_N            1532
CN_RATIO           1532
CEC_SOIL           1532
CEC_CLAY           1532
CEC_EFF            1532
TEB                1532
BSAT               1532
ALUM_SAT           1532
ESP                1532
TCARBON_EQ         1532
GYPSUM             1532
ELEC_COND          1532
dtype: int64
Final dataset saved successfully!
